In [ ]:
from ultralytics import YOLO
import torch
import matplotlib.pyplot as plt
import os
import pandas as pd

# ==========================
# VERIFICAR DISPONIBILIDAD DE GPU
# ==========================
if torch.cuda.is_available():
    dispositivo = 0
    print(" CUDA está disponible. Usando GPU.")
else:
    dispositivo = 'cpu'
    print(" CUDA no está disponible. Usando CPU.")

# ==========================
# ENTRENAMIENTO
# ==========================

# Cargar modelo base preentrenado (YOLOv11-seg)
modelo = YOLO('/workspace/yolo11x-seg.pt')

# Ejecutar entrenamiento
resultados = modelo.train(
    data='/workspace/data.yaml',
    epochs=2,
    imgsz=640,
    batch=8,
    name='yolo11x-seg_vegetation',
    project='runs/',
    optimizer='Adam',
    device=dispositivo,
    patience=5,
    lr0=0.001,
    verbose=True
)

# ==========================
# GRAFICAR ENTRENAMIENTO
# ==========================

ruta_resultados = 'runs/train/yolo11x_seg_vegetation/results.csv'

if os.path.exists(ruta_resultados):
    df = pd.read_csv(ruta_resultados)

    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(df['epoch'], df['train/box_loss'], label='Box Loss')
    plt.plot(df['epoch'], df['train/seg_loss'], label='Seg Loss')
    plt.title('Pérdida durante el entrenamiento')
    plt.xlabel('Época')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(df['epoch'], df['metrics/mAP_0.5'], label='mAP@0.5')
    plt.plot(df['epoch'], df['metrics/mAP_0.5:0.95'], label='mAP@0.5:0.95')
    plt.title('Precisión del modelo')
    plt.xlabel('Época')
    plt.ylabel('mAP')
    plt.legend()

    plt.tight_layout()
    plt.savefig('grafica_entrenamiento.png')
    plt.show()

else:
    print(" No se encontró el archivo de resultados. Asegúrate de que el entrenamiento ha comenzado.")

